# Step 1 – Consolidation

Combine all of the csv files in a single Excel workbook – one csv file per sheet. The sheet names should be the name of the csv file but without the csv extension. It’s ok if the first sheet is just a blank sheet followed by all of the data sheets for the csv files.

The Excel file should be named using just the characters in the filenames before the first hyphen. For the logs I’ve given you, the file will be called BCM.xlsx.

Each data sheet should have the following column headers in A1:C1 – datetime, scale, temperature. Here’s a little snippet of how the workbook looks after this consolidation phase:


# Your job
Well, this is a Python based course, so you’ve probably already guessed that you need to do this in Python. You cannot use Excel for any part of this other than for checking if what you are doing is working. 😊

There are a number of ways this could be done, but I will share a rough sketch of the general strategy I used and some general hints, suggestions and requirements.

At a high level, this was my approach:


# 1.	Project setup

a.	Use one of our cookiecutters to set up an appropriate folder structure.

b.	Manually copy the \logs folder into an appropriate place in your project folder. 

c.	Create a Jupyter Notebook or a .py file in which to develop your code.

d.	Put your project under version control with git.


# 2.	Create blank Excel file named BCM.xlsx

a.	Use the openpyxl library to do this with Python. The openpyxl library is already installed in the aap conda virtual environment.

b.	You can simply hard code the filename BCM.xlsx. 

c.	When you save the blank notebook using openpyxl, it will have one sheet in it (which is totally fine).


In [1]:
import openpyxl
from openpyxl import Workbook
import csv
import os
from glob import glob
from pathlib import Path
import pandas as pd

In [2]:
#!pip install XlsxWriter

In [3]:
file_path = './output/BCM.xlsx'

# 3.	Insert the contents of each csv into a new sheet in BCM.xlsx
a.	For this I used pandas and pathlib. Just used file globbing and a loop.


In [4]:
wb = Workbook()
wb.save(file_path)

b.	Read each csv into a pandas dataframe. HINT: Look at the pandas read_csv documentation to see what useful things you can accomplish during the file reading process.

c.	Inserted each dataframe into the Excel file using the appropriate dataframe method.

In [5]:
csvs_path = Path.cwd() / 'data' / 'logs'

writer = pd.ExcelWriter(file_path, engine='xlsxwriter')

# Loop over all the csv files 
for csv_fn in csvs_path.glob('*.csv'):
    # Split the filename off from csv extension. We'll use the filename
    # (without the extension) as the key in the dfs dict.
    #print(csv_fn)
    fstem = csv_fn.stem

    # Read the next csv file into a pandas DataFrame and add it to the dfs dictionary.
    df = pd.read_csv(csv_fn)
    df.columns =['Datetime', 'Temperature scale', 'Temperature']
    df['Datetime']= pd.to_datetime(df['Datetime'])
    #print(df)
    
    # Write your DataFrame to a file   
    df.to_excel(writer, fstem,index=False)


# Save the result
writer.save()

# Step 2 – Summarization
Your client now wants you to add some simple formulas to each sheet showing the minimum, maximum, and average of the temperature values. The labels should be in G2:G4 and the formulas in H2:H4. Notice, they want actual Excel formulas in H2:H4, not just computed values. Here’s a screenshot to make this clearer. Notice the nice cell formatting in column H. In addition, compute the minimum and maximum of the datetime field in rows 6 and 7.



# 4.	Add the formulas
a.	Reopened the modified Excel file with openpyxl.

b.	Loop over the appropriate sheets and create the formulas and adjacent labels.

c.	The openpyxl Tutorial at https://openpyxl.readthedocs.io/en/stable/tutorial.html has some useful information.

d.	Save the Excel file.


In [6]:
load = openpyxl.load_workbook(filename = file_path)

used_cols = ['A','B','C','G','H']

In [10]:
for sheet in load:    
    
    sheet.cell(row=2, column=7).value = 'min_temp'
    sheet.cell(row=2, column=8).value = f"=MIN(C2:C{sheet.max_row})"   
    
    sheet.cell(row=3, column=7).value = 'max_temp'
    sheet.cell(row=3, column=8).value = f"=MAX(C2:C{sheet.max_row})" 
    
    sheet.cell(row=4, column=7).value = 'mean_temp'
    sheet.cell(row=4, column=8).value = f"=ROUND(AVERAGE(C2:C{sheet.max_row}),2)"
    
    sheet.cell(row=6, column=7).value = 'min_date'
    sheet.cell(row=6, column=8).value = f"=MIN(A2:A{sheet.max_row})" 
    sheet.cell(row=6, column=8).number_format = "mm/dd/yyyy hh:mm:ss";
    
    sheet.cell(row=7, column=7).value = 'max_date'
    sheet.cell(row=7, column=8).value = f"=Max(A2:A{sheet.max_row})"
    sheet.cell(row=7, column=8).number_format = "mm/dd/yyyy hh:mm:ss";
    
    # auto resizing the columns
    for col in used_cols:
        sheet.column_dimensions[col].auto_size = True
    
load.save(filename = file_path)    

# EXTRA CREDIT:

Create a version of your stream logger code that will work even if there are files from multiple streams and where each stream gets its own Excel file. For example, instead of just filenames starting with “BCM” (the stream identifier), there were other filenames starting with “JEF”, “MAE”, and more. Make sure that each csv gets put into the correct xlsx file. 